In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display, Markdown

# --- CONFIGURACIÓN ---
# Ruta a tu base de datos
DB_PATH = 'data/database/parlamento.db'
# Número de filas a mostrar por tabla
LIMIT = 5

def display_database_overview(db_path, limit):
    """
    Se conecta a la BD y muestra las primeras N filas de cada tabla
    de manera ordenada en un notebook.
    """
    try:
        with sqlite3.connect(db_path) as conn:
            cursor = conn.cursor()
            
            # Obtener la lista de todas las tablas
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = [table[0] for table in cursor.fetchall()]
            
            display(Markdown(f"# 📊 Resumen de la Base de Datos: `{db_path}`"))
            display(Markdown(f"Mostrando las primeras **{limit} filas** de cada tabla."))
            
            for table in tables:
                if table.startswith('sqlite_'):
                    continue
                
                try:
                    # Usar pandas para leer la consulta SQL y obtener un DataFrame
                    df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT {limit}", conn)
                    
                    # Mostrar el nombre de la tabla como un título
                    display(Markdown(f"--- \n### 🏛️ Tabla: `{table}`"))
                    
                    if df.empty:
                        display(Markdown("_Esta tabla está actualmente vacía._"))
                    else:
                        # La función display() renderiza el DataFrame como una tabla HTML
                        display(df)
                        
                except Exception as e:
                    display(Markdown(f"No se pudo leer la tabla `{table}`. Error: {e}"))

    except sqlite3.Error as e:
        print(f"❌ Error de base de datos: {e}")

# --- Ejecutar la función para mostrar el resumen ---
display_database_overview(DB_PATH, LIMIT)

# 📊 Resumen de la Base de Datos: `data/database/parlamento.db`

Mostrando las primeras **5 filas** de cada tabla.

--- 
### 🏛️ Tabla: `dim_parlamentario`

,mp_uid,nombre_completo,nombre_propio,apellido_paterno,apellido_materno,genero,fecha_nacimiento,lugar_nacimiento,diputadoid,bcn_uri,url_foto,twitter_handle,sitio_web_personal,profesion,url_historia_politica,fecha_extraccion,validation_status,validation_error
0,1,María Candelaria Acevedo Sáez,María Candelaria,Acevedo,Sáez,Femenino,1958-09-12,Coronel,1096,http://datos.bcn.cl/recurso/persona/5247,https://www.bcn.cl/laborparlamentaria/imagen/1...,None,None,None,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-08-18,None,None
1,2,Eric Aedo Jeldres,Eric Mariano,Aedo,Jeldres,Masculino,1968-07-13,Los Ángeles,1097,http://datos.bcn.cl/recurso/persona/5253,https://www.bcn.cl/laborparlamentaria/imagen/1...,None,None,Administrador de Programas Sociales,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-08-18,None,None
2,3,Yovana Ahumada Palma,Yovana Alejandra,Ahumada,Palma,Femenino,1973-12-24,Antofagasta,1098,http://datos.bcn.cl/recurso/persona/5188,https://www.bcn.cl/laborparlamentaria/imagen/1...,yovyaftap2,None,Técnica agropecuaria,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-08-18,None,None
3,4,Jorge Alessandri Vergara,Jorge Iván,Alessandri,Vergara,Masculino,1979-06-08,Santiago de Chile,1009,http://datos.bcn.cl/recurso/persona/4604,https://www.bcn.cl/laborparlamentaria/imagen/1...,jalessandri,None,Abogado,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-08-18,None,None
4,5,René Alinco Bustos,René,Alinco,Bustos,Masculino,1958-06-02,,803,http://datos.bcn.cl/recurso/persona/135,https://www.bcn.cl/laborparlamentaria/imagen/1...,alinco_rene,None,Obrero de la Construcción,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-08-18,None,None


--- 
### 🏛️ Tabla: `parlamentario_aliases`

...

--- 
### 🏛️ Tabla: `interactions`

_Esta tabla está actualmente vacía._

--- 
### 🏛️ Tabla: `dim_periodo_legislativo`

,periodo_id,nombre,fecha_inicio,fecha_termino
0,1,1990-1994,1990-03-11,1994-03-10
1,2,1994-1998,1994-03-11,1998-03-10
2,3,1998-2002,1998-03-11,2002-03-10
3,4,2002-2006,2002-03-11,2006-03-10
4,5,2006-2010,2006-03-11,2010-03-10


: 

In [11]:
from src.core.context_builder import ParlamentarioContextBuilder

with ParlamentarioContextBuilder() as builder:
    # Obtener contexto completo como diccionario
    context = builder.build_complete_context(mp_uid=1)


🔨 Construyendo contexto para parlamentario con mp_uid=1
   ✓ Perfil básico: María Candelaria Acevedo Sáez
   ✓ Contexto completo generado


In [ ]:
import ollama
from src.core.context_builder import ParlamentarioContextBuilder

mp_uid = 123 

with ParlamentarioContextBuilder() as builder:
    context = builder.build_complete_context(mp_uid)
    parlamentario_context = builder.export_context_to_text(mp_uid)


def chat_with_digital_twin(user_query: str, context: str) -> str:
    # Prepara un prompt de sistema que le dé una "personalidad" al LLM
    system_prompt = (
        f"Eres un gemelo digital del diputado. Responde preguntas basándote en la "
        f"información proporcionada en el siguiente contexto. "
        f"Habla en primera persona, como si fueras el diputado. "
        f"El contexto es: {context}"
    )

    # Realiza la llamada a la API de Ollama
    response = ollama.chat(
        model='qwen3:8b', 
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_query}
        ],
        stream=False
    )

    return response['message']['content']

# Ejemplo de uso en el notebook
pregunta_usuario = "¿Cuáles son tus principales logros legislativos y en qué comisiones has participado?"
respuesta_llm = chat_with_digital_twin(pregunta_usuario, parlamentario_context)

print(f"\nPregunta del usuario: {pregunta_usuario}")
print(f"\nRespuesta del gemelo digital:\n{respuesta_llm}")

🔨 Construyendo contexto para parlamentario con mp_uid=123
   ✓ Perfil básico: Leonidas Romero Sáez
   ✓ Contexto completo generado
🔨 Construyendo contexto para parlamentario con mp_uid=123
   ✓ Perfil básico: Leonidas Romero Sáez
   ✓ Contexto completo generado
   ✓ Contexto de texto exportado a: c:\Users\benja\OneDrive\Desktop\GitHub 2025\prototype-diputados-chatbot\data\contexts\context_mp_123_20250822_234404.txt

Pregunta del usuario: ¿Cuáles son tus principales logros legislativos y en qué comisiones has participado?

Respuesta del gemelo digital:
<think>
Okay, the user is asking about my main legislative achievements and which committees I've participated in. Let me check the context provided.

First, the legislative achievements. The context says I have 20 projects as author or co-author, but none have been converted into law. Also, there are no projects in progress. Hmm, that's a bit of a problem. The user might be expecting some achievements, but the data shows none. Maybe I sh

In [4]:
!streamlit run src/app/digital_twin_app.py


"streamlit" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
